In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload
import warnings
import wrangle as w
import explore as e
warnings.simplefilter("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
coaster_df = w.wrangle_coasters()
train, validate, test = w.tvt_split(coaster_df,stratify='track_material')

In [9]:
import model as m


ImportError: cannot import name 'lmplot_kwargs' from 'custom_dtypes' (/Users/woody/codeup-data-science/individual-project/custom_dtypes.py)